In [32]:
from dotenv import load_dotenv
import os

In [28]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPEN_API_KEY')
os.environ['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')
#pinecone_env = os.getenv('PINECONE_ENV')

Embeddings

In [21]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

embedding = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=open_api_key)

In [24]:
#Embed the whole first Harry Potter Book

loader = TextLoader("harry_potter.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Created a chunk of size 1071, which is longer than the specified 1000


Vector Stores with Pinecone

In [34]:
from langchain_pinecone import PineconeVectorStore

index_name = "ai-chatbot"
docsearch = PineconeVectorStore.from_documents(docs, embedding, index_name=index_name)

Similarity Search With a Query

In [36]:
query = "What school is Harry Potter invited to attend?"

embedding_vector = embedding.embed_query(query)
# docs = docsearch.similarity_search_by_vector(embedding_vector)
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

They had reached the station. There was a train to London in five
minutes' time. Hagrid, who didn't understand "Muggle money," as he
called it, gave the bills to Harry so he could buy their tickets.

People stared more than ever on the train. Hagrid took up two seats and
sat knitting what looked like a canary-yellow circus tent.

"Still got yer letter, Harry?" he asked as he counted stitches. Harry
took the parchment envelope out of his pocket.

"Good," said Hagrid. "There's a list there of everything yeh need."

Harry unfolded a second piece of paper he hadn't noticed the night
before, and read:

HOGWARTS SCHOOL of WITCHCRAFT and WIZARDRY

UNIFORM

First-year students will require:

1. Three sets of plain work robes (black)

2. One plain pointed hat (black) for day wear

3. One pair of protective gloves (dragon hide or similar)

4. One winter cloak (black, silver fastenings)

Please note that all pupils' clothes should carry name tags

COURSE BOOKS
